In [ ]:
!pip install beautifulsoup4 
!pip install google
!pip install PyPDF2

In [ ]:
import data
import csv
import pandas as pd


In [ ]:
def readInFile(filename = 'master.csv'):
  aup_data = []
  aup_id = []
  with open(filename) as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      line_count = 0
      for row in csv_reader:
          if line_count == 0:
              print(f'Column names are {", ".join(row)}')
              line_count += 1
          else:      
              aup_data.append(row[2])
              aup_id.append(row[1])

  return aup_data, aup_id

# Url Collection

In [ ]:
# for each url we find their snapshots for the past 10 years, one for each year
def collectURL(aup_data):
  '''
  url_snapshots_dict: {
      url: (year, snapshot)
  }
  '''
  url_snapshots_dict = {}

  for url in aup_data:
    try:
      snapshots, timestamps = data.get_snapshots(url)
      snapshot_within_10_years = []
      listed_years = [] # a list that keeps the recorded years since we only want 1 snapshot each year
      for i in range(len(timestamps)):
        year = int(timestamps[i][:4])
        if year in listed_years:
          continue
        # if the timestamp is within 10 years, append corresponding snapshot
        if year >= 2013 and year <= 2023:
          snapshot_within_10_years.append((year, snapshots[i]))
          listed_years.append(year)
      url_snapshots_dict[url] = snapshot_within_10_years
    except Exception as inst: 
      print(f"Exception when dealing with {url}")
      print(inst)
      continue
  
  return url_snapshots_dict

In [ ]:
def writeToCSV(url_snapshots_dict):
  try:
    df = pd.DataFrame(url_snapshots_dict.items(), columns=['aup-url', 'snapshots'])
    df.to_csv('snapshots_from_googlesearch.csv', index=False, header=True)
    return True
  except:
    print("Error wrting to CSV")
    return False

In [ ]:
aup_data = readInFile()
url_snapshots_dict = collectURL(aup_data)
writeToCSV(url_snapshots_dict)

In [ ]:
aup_data, aup_id = readInFile()
print(len(aup_data))
print(len(aup_id))

In [ ]:
url_snapshots_dict = collectURL(aup_data)
len(url_snapshots_dict)

# Scrap AUP content

In [ ]:
len(url_snapshots_dict)

In [ ]:
# Now, we start to get AUP content using the urls
def scrapAUP(url_snapshots_dict, aup_id):
  idx = 0

  for url, tuples in url_snapshots_dict.items():
    print(f"URL currently working with {aup_id[idx]}")
    
    if len(tuples) == 0:
      print(f"{aup_id[idx]} is skipped")
      continue
    years = [t[0] for t in tuples]
    snapshots = [t[1] for t in tuples]
    for i in range(len(snapshots)):
      fname = f"snapshots/{aup_id[idx]}-{years[i]}.txt"
      data.get_aup_content(snapshots[i], fname)

    print(f"Finish with {aup_id[idx]}")
    idx += 1


In [ ]:
scrapAUP(url_snapshots_dict, aup_id)

In [ ]:
!tar -zcvf snapshots.tar.gz snapshots/ # get the data off colab

In [ ]:
!rm -rf snapshots/